In [1]:
from langchain_community.llms import Ollama
#from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
#from langchain_openai.embeddings import OpenAIEmbeddings


In [2]:
MODEL = "llama2"

In [3]:
model = Ollama(model=MODEL)

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 

In [5]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know". 

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know". \n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [6]:
chain = prompt | model | parser

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Data/Alice_wonderland.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='CHAPTER I.  \nDown the Rabbit -Hole  \nAlice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: \nonce or twice she had peeped into the book her sister was reading, but it had no pictures or \nconversations in it, “and what is the use of a book,” thou ght Alice “without pictures or \nconversations?”  \n \nSo she was considering in her own mind (as well as she could, for the hot day made her feel very \nsleepy and stupid), whether the pleasure of making a daisy -chain would be worth the trouble of \ngetting up and picking the daisies, when suddenly a White Rabbit  with pink eyes ran close by her.  \n \nThere was nothing so very remarkable in that; nor did Alice think it so very much out of the way to \nhear the Rabbit say to itself, “Oh dear! Oh dear! I shall be late!” (when she thought it over \nafterwards, it occurred to her that she ought to have wondered  at this, but at the time it all seemed \nquite natural); 

In [8]:
#Read from Text file
def read_textfile(file_path):
    with open(file_path,'r', encoding = 'utf-8') as file:
        text = file.read()
        return text

In [9]:
alice_text = read_textfile('Data/alice_wonderland_chapter1.txt')

In [10]:
from langchain_community.vectorstores import DocArrayInMemorySearch
embeddings = OllamaEmbeddings(model=MODEL)
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

c:\Users\devch\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [11]:
retriever = vectorstore.as_retriever()
#retriever.invoke("Harry Potter")

In [12]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [19]:
questions = [
    "Summarize this book. Use only the text within this context for generating the summary. Write the summary in 1000 words."
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: Summarize this book. Use only the text within this context for generating the summary. Write the summary in 1000 words.
Answer:  Alice's Adventures in Wonderland is a fantastical tale of a young girl's journey into a strange and absurd world. The story begins with Alice falling down a rabbit hole and entering a world where everything is upside down, including the Queen of Hearts who repeatedly shouts "Off with their heads!" as she orders the execution of various creatures throughout the book.

Alice encounters a variety of bizarre characters, including the March Hare, the Mad Hatter, and the Cheshire Cat, each with their own eccentricities and illogical behaviors. Throughout her adventures, Alice struggles to understand the strange rules and rhythms of this fantastical world, often finding herself in absurd and illogical situations.

Despite the absurdity of the world she finds herself in, Alice remains a relatable and endearing protagonist. Her reactions to the strange event

In [16]:
questions = [
    "Summarize this chapter"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain2.invoke({'question': question})}")
    print()

Question: Summarize this chapter


Answer: In this chapter, Alice talks to the Mock Turtle and the Gryphon about their educations. The Mock Turtle and the Gryphon have very different views on education, with the Mock Turtle believing that learning should be enjoyable and not just focused on academic subjects, while the Gryphon is more interested in practical skills like washing. Alice shares her own experiences of going to school every day and learning French and music, but she also notes that the Caterpillar's response to her repeating "You are old, Father William" was curious. The chapter ends with Alice imagining how her little sister will grow up and have children of her own, and how she will keep a loving heart towards them even as they grow older.



### Full first book


In [17]:
loader2 = PyPDFLoader("HarryPotterBook1.pdf")
pages2 = loader2.load_and_split()
pages2

[Document(page_content="1Harry Potter and the Sorcerer's Stone\nCHAPTER ONE\nTHE BOY WHO LIVEDMr. and Mrs. Dursley, of number four, Privet Drive, were proud to say\nthat they were perfectly normal, thank you very much. They were the last\npeople you'd expect to be involved in anything strange or mysterious,because they just didn't hold with such nonsense.\nMr. Dursley was the director of a firm called Grunnings, which made\ndrills. He was a big, beefy man with hardly any neck, although he did\nhave a very large mustache. Mrs. Dursley was thin and blonde and had\nnearly twice the usual amount of neck, which came in very useful as shespent so much of her time craning over garden fences, spying on theneighbors. The Dursleys had a small son called Dudley and in theiropinion there was no finer boy anywhere.\nThe Dursleys had everything they wanted, but they also had a secret, and\ntheir greatest fear was that somebody would discover it. They didn'tthink they could bear it if anyone found ou

In [18]:
vectorstore2 = DocArrayInMemorySearch.from_documents(pages2, embedding=embeddings)

KeyboardInterrupt: 

In [ ]:
retriever2 = vectorstore2.as_retriever()

In [ ]:
chain3 = (
    {
        "context": itemgetter("question") | retriever2,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [ ]:
questions = [
    "Summarize this book chapter-wise. Each chapter should be summarized in 5 to 6 lines"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain3.invoke({'question': question})}")
    print()

Question: Summarize this book chapter-wise. Each chapter should be summarized in 5 to 6 lines
Answer: Sure, here is a summary of each chapter in Harry Potter and the Philosopher's Stone:

Chapter 1: The boy who lived - Introduces Harry Potter, his parents, and how he is famous in the wizarding world after surviving a killing curse from the Dark Lord.

Chapter 2: A letter from Hogwarts - Explains how Harry receives a letter from Hogwarts School of Witchcraft and Wizardry and how he finds out he is a wizard.

Chapter 3: The Hogwarts Express and Hogsmeade - Describes Harry's journey to Hogwarts on the Hogwarts Express, where he meets Ron Weasley and Hermione Granger, and their adventures in Hogsmeade.

Chapter 4: The Potions Master - Introduces Professor Snape, the Potions master at Hogwarts, who is portrayed as a mean and unfair teacher.

Chapter 5: The Quidditch Match - Describes Harry's excitement for the Quidditch match between Gryffindor and Slytherin, and how he helps his friends wi